### Imports

In [3]:
from random import random
from random import seed
from random import randint
import os
import numpy as np

### Score Metrics: TOS and ADS

In [6]:
def score_metrics(scores, nbooks, signup, dailyLimit, daysLeft): 
    # total offered score (TOS): the max score that the library can provide in the remaining days including signup
    # average daily score (ADS): approx daily offering based on TOS
    # scores: sorted list of remaining books
    # nbooks: number of books
    # signup: signup time in days
    # dailylLimit: number of books scanned daily
    # daysLeft: number of remaining days
    
    # reqDays: days required to scan all books
    # scanDays: days available to scan all books
    # nleftover: extra books that cannot be scanned
    
    if signup >= daysLeft:
        return 0,0 
    
    totalScore = sum(scores)
    reqDays = nbooks/dailyLimit
    scanDays = daysLeft - signup
    
    nleftover = nbooks - (dailyLimit * scanDays)
    
    if nleftover < 0:
        nleftover = 0
    
    tos = sum(scores[:len(scores)-nleftover])
    
    if scanDays >= reqDays:
        ads = tos/(reqDays+signup)
    else:
        ads = tos/(scanDays+signup)
        
    
    return tos,ads    

### Credit Metrics

In [7]:
def credit_metrics(tos,ads,signup):
    # TOS: total offered score
    # ADS: average daily score
    # signup: signup time in days
    
    credit = (tos*ads)/signup
    return credit

### Class Definitions

In [4]:
class Library():
    
    def __init__(self,**kwargs):
        self.name = kwargs.pop('name')
        self.signup = kwargs.pop('signup')
        self.dailyLimit = kwargs.pop('dailyLimit')
        self.nbooks = kwargs.pop('nbooks')
        self.scores = kwargs.pop('scores')
        self.indexes = kwargs.pop('indexes')
        self.totalScore = sum(self.scores)
        self.tos = 0
        self.ads = 0        
        self.credit = 0
        return
    
    def scoreIndexMap(self):
        s = sorted(zip(self.scores,self.indexes))
        self.scores,self.indexes = map(list, zip(*s))
        del(s)
        return
    
    def getMetrics(self,**kwargs):
        daysLeft = kwargs.pop('daysLeft')
        self.tos,self.ads = score_metrics(self.scores, self.nbooks, self.signup, self.dailyLimit, daysLeft)
        self.credit = credit_metrics(self.tos,self.ads,self.signup)
        return
    
    def showDetails(self,flag='v'):
        if flag == 'v':
            print('{}:\n nbooks: {}\n Sum: {}\n Daily Limit: {}\n Signup: {}\n TOS: {}\n ADS: {:.2f}\n Credit: {:.2f}\n Scores: {}\n Indexes: {}\n\n'.format(self.name,self.nbooks,self.totalScore,self.dailyLimit,self.signup,self.tos,self.ads,self.credit,self.scores,self.indexes))
        elif flag == 'm':
            print('{}:\n TOS: {}\n ADS: {:.2f}\n Signup: {}\n Credit: {:.2f}\n Scores: {}\n Indexes: {}\n\n'.format(self.name,self.tos,self.ads,self.signup,self.credit,self.scores,self.indexes))

def outputLibrary():
    
    def __init__(self,index,bookList):
        self.index = index
        self.bookList = bookList
        self.nbooks = len(bookList)

### Testing Functions

In [32]:
def generateBookScores(totalScore, nbooks):
    # totalScore: the max score of book list
    # nbooks: the number of books in the list
    
    # scoreList: final scores of argument specifications
    
    scoreList = []
    runningTotal = totalScore
    runningIndex = nbooks
    
    for i in range(nbooks):
        initialLimit = runningTotal - runningIndex
        score = randint(1,initialLimit)
        scoreList.append(score)
        runningTotal -= score
        runningIndex -= 1
    
    extra = totalScore - sum(scoreList)
    scoreList[randint(0,nbooks-1)] += extra
    
    scoreList.sort(reverse=True)
    return scoreList

scores = generateBookScores(300,12)
print(' Score List: {}\n Sum: {}\n'.format(scores,sum(scores)))

 Score List: [225, 36, 20, 11, 1, 1, 1, 1, 1, 1, 1, 1]
 Sum: 300



### Testing Zone

In [ ]:

libraryList = []

libraryList.append(Library(nbooks = 12, scores=generateBookScores(30,12), signup=30, dailyLimit=1, name='L1'))
libraryList.append(Library(nbooks = 12, scores=generateBookScores(40,12), signup=50, dailyLimit=1, name='L2'))
libraryList.append(Library(nbooks = 12, scores=generateBookScores(60,12), signup=60, dailyLimit=1, name='L3'))

D = 100
for daysLeft in range(D,1,-10):
    flag = 1
    if daysLeft != D:
        for lib in libraryList:
            if daysLeft == lib.signup or daysLeft == (D-lib.signup):
                flag = 0
    
    if flag == 1 and daysLeft != D:
        continue
            
    print(f' t = {D-daysLeft} \t Days left: {daysLeft}\n')
    for lib in libraryList:
        lib.getMetrics(daysLeft=daysLeft)
        lib.showDetails('m')

    print('_____________________________________________________________\n')        

### Input 

In [9]:
# File to be read
input_filename = 'a_example.txt'

# File read and format
f = open(input_filename,'r')
data = f.read().split('\n')
for i in range(len(data)):
    data[i] = list(map(int,data[i].split()))
data = data[:-1]

# Global variables
B = data[0][0]                    # Total number of books
NSEEN = [x for x in range(B)]     # Indexes of books not seen
SEEN = []                         # Indexes of books already seen
S = data[1]                       # Scores of each books
score_map = lambda i: S[i]

NL = data[0][1]                  # Number of libraries
D = data[0][2]                   # Number of days

# Library objects
libraryList = []
for i in range(2,len(data),2):
    scores = list(map(score_map,data[i+1]))
    libraryList.append(Library(nbooks=data[i][0], 
                               scores=scores, 
                               indexes = data[i+1],
                               signup=data[i][1], 
                               dailyLimit=data[i][2], 
                               name='L{:.0f}'.format((i/2)-1)))
    
f.close()

print(f' B: {B}\n S: {S}\n NSEEN:{NSEEN}\n SEEN:{SEEN}\n\n NL: {NL}\n D: {D}\n\n Library Data:\n')
for lib in libraryList:
    lib.scoreIndexMap()
    lib.getMetrics(daysLeft=D)
    lib.showDetails()



 B: 6
 S: [1, 2, 3, 6, 5, 4]
 NSEEN:[0, 1, 2, 3, 4, 5]
 SEEN:[]

 NL: 2
 D: 7

 Library Data:

L0:
 nbooks: 5
 Sum: 17
 Daily Limit: 2
 Signup: 2
 TOS: 17
 ADS: 3.78
 Credit: 32.11
 Scores: [1, 2, 3, 5, 6]
 Indexes: [0, 1, 2, 4, 3]


L1:
 nbooks: 4
 Sum: 14
 Daily Limit: 1
 Signup: 3
 TOS: 14
 ADS: 2.00
 Credit: 9.33
 Scores: [1, 3, 4, 6]
 Indexes: [0, 2, 5, 3]




### Output

In [10]:
# Filename
output_filename = input_filename[:-4] + '_result.txt'

string_lambda = lambda x: 

# File write
f = open(output_filename,'w',newline='\n')
outputLibraryList = [] # list of outputLibrary objects
f.write(f'{len(outputLibraryList)}\n')
for lib in outputLibraryList:
    f.write(f'{lib.index} {lib.nbooks}\n')
    f.write(' '.join(str(x) for x in lib.bookList))
    f.write('\n')
f.close()

# Done
print(' Result saved\n')

a_example_result.txt
